In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())  # Check if GPU is available

2.5.1+cu121
True


In [4]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [5]:
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

In [6]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

trainloader = DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=128, shuffle=False, num_workers=2)

100%|██████████| 170M/170M [00:02<00:00, 58.0MB/s] 


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [7]:
import torch.nn as nn
import torchvision.models as models

model = models.resnet18(weights=None, num_classes=10)  # CIFAR-10 has 10 classes

In [8]:
model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
model.maxpool = nn.Identity()  # Remove the initial max pooling layer

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print(torch.cuda.is_available())  # Should print True if GPU is available


True


In [10]:
criterion = nn.CrossEntropyLoss()

In [11]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)

In [12]:
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

In [13]:
num_epochs = 50

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for i, (inputs, labels) in enumerate(trainloader):
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if i % 100 == 99:  # Print every 100 mini-batches
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(trainloader)}], Loss: {running_loss/100:.4f}')
            running_loss = 0.0

    scheduler.step()

Epoch [1/50], Step [100/391], Loss: 2.4860
Epoch [1/50], Step [200/391], Loss: 1.8677
Epoch [1/50], Step [300/391], Loss: 1.7054
Epoch [2/50], Step [100/391], Loss: 1.5000
Epoch [2/50], Step [200/391], Loss: 1.3944
Epoch [2/50], Step [300/391], Loss: 1.3052
Epoch [3/50], Step [100/391], Loss: 1.1294
Epoch [3/50], Step [200/391], Loss: 1.0371
Epoch [3/50], Step [300/391], Loss: 0.9768
Epoch [4/50], Step [100/391], Loss: 0.8738
Epoch [4/50], Step [200/391], Loss: 0.8534
Epoch [4/50], Step [300/391], Loss: 0.8016
Epoch [5/50], Step [100/391], Loss: 0.7310
Epoch [5/50], Step [200/391], Loss: 0.7051
Epoch [5/50], Step [300/391], Loss: 0.6627
Epoch [6/50], Step [100/391], Loss: 0.6179
Epoch [6/50], Step [200/391], Loss: 0.6209
Epoch [6/50], Step [300/391], Loss: 0.6006
Epoch [7/50], Step [100/391], Loss: 0.5680
Epoch [7/50], Step [200/391], Loss: 0.5613
Epoch [7/50], Step [300/391], Loss: 0.5488
Epoch [8/50], Step [100/391], Loss: 0.5052
Epoch [8/50], Step [200/391], Loss: 0.5346
Epoch [8/50

In [14]:
torch.save(model.state_dict(), 'resnet18_cifar10.pth')

In [15]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on test set: {100 * correct / total:.2f}%')

Accuracy on test set: 93.35%
